<a href="https://www.kaggle.com/code/siddp6/flower-images-classification?scriptVersionId=139093022" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Crafting an AI Application

The future sees AI algorithms integrated into everyday applications. For instance, think of an image classifier in a smartphone app, powered by a deep learning model trained on vast image datasets. This integration of models into applications will become a staple in software development.

This project focuses on training an image classifier to recognize flower species. Imagine a camera app identifying flowers. We'll use a dataset of [102 flower categories](https://www.kaggle.com/datasets/siddp6/flower-dataset). The project involves:

- Loading and preprocessing the image dataset
- Training the classifier
- Using it for image predictions

Each step will be guided, implemented in Python.

This project will equip you to build classifiers for any labeled image set. Apply these skills to create innovative applications, like identifying car models from pictures. Let's start by importing required packages, adhering to the practice of placing imports at the code's outset.


In [1]:
# Importing required packages

import os  # Operating system functions
import json  # For working with JSON files
import torch  # PyTorch library

import torch.nn.functional as F  # Functional module of PyTorch's neural network
import numpy as np  # Numerical computing library
import matplotlib.pyplot as plt  # Plotting library

from torch import nn  # Neural network module of PyTorch
from torch import optim  # Optimization algorithms
from collections import OrderedDict  # For creating ordered dictionaries
from torchvision import datasets, transforms, models  # PyTorch's vision library
from PIL import Image  # Python Imaging Library

In [2]:
# Set up matplotlib for inline plotting with high-resolution figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
# Hyperparameters
batch_size = 64  # Number of samples per batch during training
epochs = 10  # Number of times to iterate over the entire dataset
learning_rate = 0.001  # Learning rate for updating model parameters
criterion = nn.NLLLoss()  # Negative Log Likelihood loss

# Other constants
input_size = 25088  # Input size of the network (flattened image size)
output_size = 102  # Number of possible output classes (flower species)
hidden_layers = 512  # Number of units in the hidden layers
drop_rate = 0.2  # Probability of dropout during training

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Using GPU if available, else CPU

## Loading the Data

In this section, you will utilize `torchvision` to load the data. The dataset is provided either alongside this notebook or can be [downloaded here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). It's divided into three main parts: training, validation, and testing.

For the training data, various transformations will be applied, including random scaling, cropping, and flipping. These transformations enhance the network's ability to generalize and improve its overall performance. Moreover, it's crucial to resize the input data to 224x224 pixels to comply with the requirements of the pre-trained networks.

The validation and testing sets play a role in assessing the model's performance on unseen data. Consequently, no scaling or rotation transformations are applied here. Instead, images need to be resized and cropped to the appropriate dimensions.

The pre-trained networks that will be utilized were initially trained on the ImageNet dataset, where normalization was performed separately for each color channel. For all three datasets (training, validation, and testing), it is essential to normalize the image means and standard deviations to match the expectations of the network. The mean values are set to `[0.485, 0.456, 0.406]`, and the standard deviations to `[0.229, 0.224, 0.225]`, which are calculated from the ImageNet images. By using these values, each color channel is centered around 0 and ranges from -1 to 1.


In [4]:
# Set the directory paths for training, validation, and testing data
data_dir = '/kaggle/input/flower-dataset/flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [5]:
# Creating a normalization transform that adjusts image values to have a standard mean and deviation
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],  # Set the mean values for each color channel
    std=[0.229, 0.224, 0.225]  # Set the standard deviation values for each color channel
)

# Defining transformation pipelines for different datasets (train, val, test)
data_transforms = {}

# For training data: Randomly crop and resize to 224x224, then convert to tensor and normalize
data_transforms['train'] = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Crop and resize randomly
    transforms.ToTensor(),  # Convert to tensor
    normalize  # Normalize using the defined mean and std
])

# For validation data: Resize to 256x256, center crop to 224x224, then convert to tensor and normalize
data_transforms['val'] = transforms.Compose([
    transforms.Resize(256),  # Resize
    transforms.CenterCrop(224),  # Center crop
    transforms.ToTensor(),  # Convert to tensor
    normalize  # Normalize using the defined mean and std
])

# For test data: Resize to 256x256, center crop to 224x224, then convert to tensor and normalize
data_transforms['test'] = transforms.Compose([
    transforms.Resize(256),  # Resize
    transforms.CenterCrop(224),  # Center crop
    transforms.ToTensor(),  # Convert to tensor
    normalize  # Normalize using the defined mean and std
])

# Load the datasets using ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),  # Load train data
    'val': datasets.ImageFolder(valid_dir, transform=data_transforms['val']),  # Load validation data
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])  # Load test data
}

# Create dataloaders for the datasets
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size, shuffle=True),  # Train dataloader
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size, shuffle=True),  # Validation dataloader
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=32, shuffle=True)  # Test dataloader
}

In [6]:
# A mapping of category names to labels
cat_to_name = {"21": "fire lily", "3": "canterbury bells", "45": "bolero deep blue", "1": "pink primrose", "34": "mexican aster", "27": "prince of wales feathers", "7": "moon orchid", "16": "globe-flower", "25": "grape hyacinth", "26": "corn poppy", "79": "toad lily", "39": "siam tulip", "24": "red ginger", "67": "spring crocus", "35": "alpine sea holly", "32": "garden phlox", "10": "globe thistle", "6": "tiger lily", "93": "ball moss", "33": "love in the mist", "9": "monkshood", "102": "blackberry lily", "14": "spear thistle", "19": "balloon flower", "100": "blanket flower", "13": "king protea", "49": "oxeye daisy", "15": "yellow iris", "61": "cautleya spicata", "31": "carnation", "64": "silverbush", "68": "bearded iris", "63": "black-eyed susan", "69": "windflower", "62": "japanese anemone", "20": "giant white arum lily", "38": "great masterwort", "4": "sweet pea", "86": "tree mallow", "101": "trumpet creeper", "42": "daffodil", "22": "pincushion flower", "2": "hard-leaved pocket orchid", "54": "sunflower", "66": "osteospermum", "70": "tree poppy", "85": "desert-rose", "99": "bromelia", "87": "magnolia", "5": "english marigold", "92": "bee balm", "28": "stemless gentian", "97": "mallow", "57": "gaura", "40": "lenten rose", "47": "marigold", "59": "orange dahlia", "48": "buttercup", "55": "pelargonium", "36": "ruby-lipped cattleya", "91": "hippeastrum", "29": "artichoke", "71": "gazania", "90": "canna lily", "18": "peruvian lily", "98": "mexican petunia", "8": "bird of paradise", "30": "sweet william", "17": "purple coneflower", "52": "wild pansy", "84": "columbine", "12": "colt's foot", "11": "snapdragon", "96": "camellia", "23": "fritillary", "50": "common dandelion", "44": "poinsettia", "53": "primula", "72": "azalea", "65": "californian poppy", "80": "anthurium", "76": "morning glory", "37": "cape flower", "56": "bishop of llandaff", "60": "pink-yellow dahlia", "82": "clematis", "58": "geranium", "75": "thorn apple", "41": "barbeton daisy", "95": "bougainvillea", "43": "sword lily", "83": "hibiscus", "78": "lotus lotus", "88": "cyclamen", "94": "foxglove", "81": "frangipani", "74": "rose", "89": "watercress", "73": "water lily", "46": "wallflower", "77": "passion flower", "51": "petunia"}

# Building and Training the Classifier

With the prepared data in place, it's time to construct and train the classifier. In the following steps, you'll leverage a pre-trained model from `torchvision.models` to extract image features. The goal is to create and train a novel feed-forward classifier using these extracted features.

Here's what you'll be doing:

* **Load a Pre-trained Network:** Begin by selecting a pre-trained network from the [`torchvision.models`](http://pytorch.org/docs/master/torchvision/models.html) module. If you're looking for a good starting point, the VGG networks are effective and easy to utilize.

* **Design the Classifier:** Define a fresh, untrained feed-forward network to function as a classifier. This classifier should incorporate Rectified Linear Unit (ReLU) activations and dropout for regularization.

* **Train the Classifier Layers:** Employ backpropagation to train the classifier layers. To extract meaningful features, use the pre-trained network's weights.

* **Track Loss and Accuracy:** Throughout training, monitor the loss and accuracy metrics on the validation dataset. These metrics will help you identify the optimal hyperparameters.

When conducting training, remember to update solely the weights of the feed-forward network. If everything is implemented correctly, you should achieve a validation accuracy exceeding 70%. Experiment with various hyperparameters—such as learning rate, classifier units, and epochs—to uncover the best model configuration. Keep track of these successful hyperparameters, as they'll serve as default values for the subsequent parts of the project.


In [7]:
# Load a pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Iterate through model parameters and set their 'requires_grad' attribute to False
# This freezes the parameters, preventing them from being updated during training
for param in model.parameters():
    param.requires_grad = False

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 56.4MB/s] 


In [8]:
import torch.nn as nn
import torch.nn.functional as F

class MyNetwork(nn.Module):
    """
    A simple feedforward neural network model.

    Args:
        input_size (int): Number of input features.
        hidden_units (int): Number of hidden units in the layers.
        drop_rate (float): Dropout rate for regularization.
        output_size (int): Number of output classes.

    Attributes:
        layer1 (nn.Linear): First fully connected layer.
        layer2 (nn.Linear): Second fully connected layer.
        layer3 (nn.Linear): Third fully connected layer.
        dropout (nn.Dropout): Dropout layer for regularization.
        output (nn.LogSoftmax): Log-softmax activation for classification.
    """
    def __init__(self, input_size, hidden_units, drop_rate, output_size):
        super().__init__()

        self.layer1 = nn.Linear(input_size, hidden_units)
        self.layer2 = nn.Linear(hidden_units, hidden_units // 2)
        self.layer3 = nn.Linear(hidden_units // 2, output_size)

        self.dropout = nn.Dropout(drop_rate)

        self.output = nn.LogSoftmax(dim=1)

    def forward(self, x):
        """
        Forward pass through the network.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor after passing through the network.
        """
        out = self.layer1(x)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.layer2(out)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.layer3(out)

        out = self.output(out)

        return out

In [9]:
# Update the classifier of the model with a custom network defined by MyNetwork,
# using the specified input size, hidden layers, dropout rate, and output size
model.classifier = MyNetwork(input_size, hidden_layers, drop_rate, output_size)

In [10]:
import torch

def get_train_validation(model, device, dataloaders, criterion):
    """
    Calculate loss and accuracy on the validation dataset.

    Args:
        model (torch.nn.Module): The neural network model.
        device (torch.device): The device (CPU or GPU) for computation.
        dataloaders (dict): Dictionary containing data loaders for training and validation.
        criterion (torch.nn.Module): The loss function.

    Returns:
        tuple: A tuple containing loss and accuracy values.
    """
    loss, accuracy = 0, 0

    model.to(device)  # Move the model to the specified device
    validloader = dataloaders["val"]  # Get the validation data loader

    for images, labels in validloader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)  # Generate predictions
        loss += criterion(output, labels).item()  # Calculate loss
        ps = torch.exp(output)
        equity = labels == ps.max(dim=1)[1]  # Check if predictions are correct
        accuracy += equity.type(torch.FloatTensor).mean()  # Calculate accuracy

    return loss, accuracy

In [11]:
def train_network():
    """
    Train the network using the specified data loaders and settings.

    Returns:
        torch.nn.Module: The trained model.
    """
    trainloader = dataloaders["train"]  # Get the training data loader
    optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)  # Use Adam optimizer for model's classifier parameters

    model.to(device)  # Move the model to the appropriate device (CPU or GPU)

    steps = 0  # Initialize the steps counter

    for epoch in range(epochs):  # Loop through the specified number of epochs
        training_loss = 0  # Initialize training loss for this epoch
        for images, labels in trainloader:  # Loop through batches of training data
            steps += 1  # Increment the steps counter
            images, labels = images.to(device), labels.to(device)  # Move images and labels to the device
            optimizer.zero_grad()  # Clear previous gradients
            output = model.forward(images)  # Perform a forward pass
            Loss = criterion(output, labels)  # Calculate the loss

            Loss.backward()  # Perform backpropagation
            optimizer.step()  # Update model's weights

            training_loss += Loss.item()  # Accumulate training loss

        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation
            loss, accuracy = get_train_validation(model, device, dataloaders, criterion)  # Calculate validation loss and accuracy

        print(
            "Epoch: {}/{}\nTraining Loss: {:.4f}\nValidation Loss: {:.4f}\nValidation Accuracy: {:.3f}%\n\n".format(
                epoch + 1,
                epochs,
                training_loss / len(dataloaders["train"]),
                loss / len(dataloaders["val"]),
                accuracy / len(dataloaders["val"]) * 100,
                )
            ) # Print epoch summary

        model.train()  # Set the model back to training mode

    return model  # Return the trained model

In [ ]:
# Trains a neural network and returns the trained model
trained_model = train_network()

Epoch: 1/10
Training Loss: 2.7753
Validation Loss: 1.0367
Validation Accuracy: 73.716%




## Evaluating Your Network

Testing your trained network on unseen data is a vital step. This test should involve images that your network has never encountered during training or validation. Doing so provides an accurate gauge of your model's performance with entirely new images. Just as you did with validation, feed the test images through your network and calculate accuracy. A well-trained model should achieve approximately 70% accuracy on the test set.


In [ ]:
def get_test_validation(model, dataloaders, device):
    """
    Evaluate model accuracy on the test data.

    Args:
        model (torch.nn.Module): The neural network model.
        dataloaders (dict): Dictionary containing dataloaders for different datasets.
        device (str): Device to perform computations on (e.g., 'cpu' or 'cuda').

    Returns:
        tuple: A tuple containing accuracy and total data size.
    """
    accuracy, data_size = 0, 0

    model.to(device)  # Move the model to the specified device

    model.eval()  # Set the model in evaluation mode
    for images, labels in dataloaders['test']:
        images, labels = images.to(device), labels.to(device)  # Move data to the same device
        output = model.forward(images)  # Forward pass
        ps = torch.exp(output)  # Compute the softmax probabilities
        equity = labels == ps.max(dim=1)[1]  # Check if predictions match the labels
        data_size += labels.size(0)  # Accumulate the total number of data points
        accuracy += equity.type(torch.FloatTensor).sum().item()  # Accumulate correct predictions

    return accuracy, data_size  # Return accuracy and total data size

In [ ]:
# Use the torch.no_grad() context to disable gradient computation
with torch.no_grad():
    # Call the get_test_validation() function to get accuracy and total
    accuracy, total = get_test_validation(model, dataloaders, device)

# Print the accuracy as a percentage
print('Accuracy: {}%\n'.format(100 * accuracy / total))

## Save the checkpoint

After training your network, it's important to save the model for future use in making predictions. Additionally, consider saving other necessary information like the class-to-index mapping, which can be obtained from an image dataset using `image_datasets['train'].class_to_idx`. You can attach this mapping to the model as an attribute, making inference easier in the future:

```python
model.class_to_idx = image_datasets['train'].class_to_idx
```

In [ ]:
def save_model(save_dir):
    """
    Save the trained model checkpoint.

    Args:
        save_dir (str): Path to the directory where the checkpoint will be saved.
    """
    checkpoint = {
        "architecture": 'vgg16',  # The architecture used for the model
        "state_dict": model.state_dict(),  # Save the model's learned weights
        "class_to_idx": image_datasets['train'].class_to_idx,  # Mapping of classes to indices
        'model': models.vgg16(pretrained=True),  # Load a pre-trained VGG16 model
        'classifier': model.classifier  # Save the custom classifier part of the model
    }

    torch.save(checkpoint, save_dir)  # Save the checkpoint to the specified directory

In [ ]:
save_model('checkpoint.pth')  # Save the model to a file named 'checkpoint.pth'

## Loading the checkpoint

Now is a good time to create a function that can load a checkpoint and reconstruct the model. This will allow you to resume working on your project later without the need to retrain the network from scratch.


In [ ]:
def get_loaded_model(checkpoint):
    """
    Load a pre-trained model from a checkpoint.

    Args:
        checkpoint (str): The path to the checkpoint file.

    Returns:
        torch.nn.Module: The loaded pre-trained model.
    """
    # Load the checkpoint
    checkpoint = torch.load(checkpoint)

    # Extract necessary components from the checkpoint
    model = checkpoint['model']
    model.class_to_idx = checkpoint['class_to_idx']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])

    # Freeze model parameters to prevent gradient updates during inference
    for param in model.parameters():
        param.requires_grad = False

    return model

In [ ]:
# Load a previously saved model from a checkpoint file
model = get_loaded_model('checkpoint.pth')

# Inference for Classification

In this section, you will create a function to use a trained network for inference. The goal is to pass an image through the network and predict the class of the flower shown in the image. To accomplish this, you'll define a function called `predict` that takes an image and a model as inputs. The function will then return the top *K* most likely classes along with their corresponding probabilities. The usage will look like this:

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [0.01558163, 0.01541934, 0.01452626, 0.01443549, 0.01407339]
> ['70', '3', '45', '62', '55']
```
First, you need to handle preprocessing the input image to make it compatible with your network.

## Image Preprocessing

1. Utilize the PIL library to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)).
2. It's recommended to create a function that preprocesses the image, adapting it for use as input in your model. This function should process the images in the same manner they were treated during training.
3. Start by resizing the images, making the shortest side 256 pixels while preserving the aspect ratio. This can be achieved using the `thumbnail` or `resize` methods.
4. Following resizing, crop the central 224x224 portion of the image.
5. Image color channels are usually encoded as integers ranging from 0 to 255. However, the model expects values between 0 and 1 in float format. You will need to perform this conversion. You can obtain a Numpy array from a PIL image using `np_image = np.array(pil_image)`.
6. Just like during training, the network anticipates images to be normalized in a specific way. The means are `[0.485, 0.456, 0.406]`, and the standard deviations are `[0.229, 0.224, 0.225]`. Subtract the means from each color channel and then divide by the standard deviation.
7. Lastly, PyTorch expects the color channel to be the first dimension, while it's the third dimension in both the PIL image and the Numpy array. You can rearrange dimensions using `ndarray.transpose`. The color channel needs to be the first dimension while maintaining the order of the other two dimensions.


To verify your progress, the function provided below converts a PyTorch tensor and visualizes it within the notebook. If your `process_image` function is functioning correctly, passing the output through this function should yield the original image (with the exception of cropped sections).


In [ ]:
def process_image(image):
    """
    Process an image for neural network input.

    Args:
        image (str): File path to the image.

    Returns:
        torch.Tensor: Transformed image as a PyTorch tensor.
    """
    image = Image.open(image)  # Open the image using PIL

    # Define a sequence of transformations to process the image
    image_transform = transforms.Compose(
        [
            transforms.Resize(255),  # Resize the image to 255x255 pixels
            transforms.CenterCrop(224),  # Crop the image to 224x224 pixels, centered
            transforms.ToTensor(),  # Convert the image to a PyTorch tensor
            normalize  # Apply normalization (assuming 'normalize' is defined elsewhere)
        ]
    )

    return image_transform(image)  # Apply the defined transformations and return the processed image

In [ ]:
def imshow(image, title=None):
    """
    Display an image using matplotlib.

    Args:
        image (torch.Tensor): The image to display, should be in tensor format.
        title (str, optional): Title for the displayed image. Default is None.
    """
    fig, ax = plt.subplots()  # Create a new figure and axis
    
    # Transpose the image from tensor format (C, H, W) to (H, W, C)
    image = image.numpy().transpose((1, 2, 0))
    
    mean = np.array([0.485, 0.456, 0.406])  # Mean values for normalization
    std = np.array([0.229, 0.224, 0.225])   # Standard deviation values for normalization
    
    # Normalize and clip the image pixel values to the range [0, 1]
    image = (std * image) + mean
    image = np.clip(image, 0, 1)
    
    if title:
        ax.set_title(title)  # Set the title if provided
    ax.imshow(image)  # Display the image

In [ ]:
# Display the processed image using the process_image function
imshow(process_image('/kaggle/input/flower-dataset/flowers/test/12/image_03994.jpg'))

## Class Prediction

To make predictions with your model using images in the correct format, you'll need to implement a prediction function. It's a common practice to predict the top 5 (often referred to as top-$K$) most probable classes. Here's how you can achieve this:

1. Calculate class probabilities.
2. Find the top $K$ largest values among the probabilities.

You can use the [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk) method to get the top $K$ probabilities and their corresponding indices. After obtaining the indices, you'll need to convert them into actual class labels using the `class_to_idx` mapping. If you've added this mapping to the model or used an `ImageFolder` to load the data ([see here](#Save-the-checkpoint)), you can perform this conversion. Remember to invert the dictionary so that you can map from index to class as well.

Your prediction function should take the path to an image and a model checkpoint as input, and return the predicted probabilities and class labels. Here's a sample code snippet:

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def get_prediction(image_path, model, device, topk=5):
    """
    Get predictions for an image using a trained model.

    Args:
        image_path (str): Path to the input image.
        model (torch.nn.Module): The trained neural network model.
        device (str): Device to perform computations on (e.g., 'cuda' or 'cpu').
        topk (int): Number of top predictions to return.

    Returns:
        tuple: A tuple containing arrays of predicted probabilities and corresponding class labels.
    """
    # Process the image and prepare it for prediction
    image = process_image(image_path).unsqueeze(0).float()
    
    # Move model and image to the specified device
    model, image = model.to(device), image.to(device)
    
    # Set the model in evaluation mode
    model.eval()
    
    with torch.no_grad():
        # Perform forward pass through the model
        results = torch.exp(model.forward(image))
        
        # Get the topk predictions
        result_top_k = results.topk(topk)
        
        # Extract predicted probabilities and class indices
        probs, classes = result_top_k[0].data.cpu().numpy()[0], result_top_k[1].data.cpu().numpy()[0]
        
        # Convert class indices to actual class labels using class_to_idx mapping
        idx_to_class = {key: value for value, key in model.class_to_idx.items()}
        classes = [idx_to_class[classes[i]] for i in range(classes.size)]

    return probs, classes

In [ ]:
# Get predictions using the get_prediction function
probs, predict_classes = get_prediction(
    '/kaggle/input/flower-dataset/flowers/test/12/image_03994.jpg', model, device
)

# Print the predicted classes
print(predict_classes)

# Print the predicted probabilities
print(probs)

## Sanity Checking

After training a model and generating predictions, it's important to perform a sanity check to ensure the predictions make sense. Even if the testing accuracy is high, it's always a good practice to verify for any possible bugs or anomalies. To achieve this, you can utilize the `matplotlib` library to create a bar graph showing the probabilities of the top 5 predicted classes, alongside the input image. The visualization might resemble the following example:


To convert from the class integer encoding to actual flower names, utilize the `cat_to_name.json` file that should have been loaded earlier in the notebook. Additionally, to display a PyTorch tensor as an image, you can utilize the `imshow` function defined above.


In [ ]:
def view_classify(image_path, probs, classes):
    """
    Display an image with its predicted class probabilities.

    Args:
        image_path (str): Path to the image file.
        probs (list): List of predicted class probabilities.
        classes (list): List of predicted class indices.

    Returns:
        None
    """
    # Get the names of the predicted classes using cat_to_name dictionary
    name = [cat_to_name[i] for i in classes]
    
    # Display the processed image using imshow
    imshow(process_image(image_path), cat_to_name[classes[0]])

    # Create a figure with two subplots (image and graph)
    _, (ax_img, ax_gph) = plt.subplots(nrows=2)
    
    # Turn off the axis for the image subplot
    ax_img.axis('off')
    
    # Create a horizontal bar graph for class probabilities
    ax_gph.barh(np.arange(len(name)), probs)
    
    # Set y-ticks to class indices and labels to class names
    ax_gph.set_yticks(np.arange(len(name)))
    ax_gph.set_yticklabels(name)
    
    # Invert the y-axis for a better display
    ax_gph.invert_yaxis()

In [ ]:
# Call the function 'view_classify' to visualize classification results for a specific image
view_classify('/kaggle/input/flower-dataset/flowers/test/12/image_03994.jpg', probs, predict_classes)
